<a href="https://colab.research.google.com/github/gfrts/calculadora_aluguel/blob/main/web_scrapping_viva_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

     |████████████████████████████████| 911kB 7.6MB/s 


In [2]:
!pip install webdriver_manager

In [3]:
!pip install kora -q

     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 


In [4]:
import pandas as pd
from selenium import webdriver
from kora.selenium import wd
from time import sleep
from tqdm import tqdm

In [46]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [20]:
#links e classes
VIVA_REAL_LINK = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/'
RESULT_LIST_CLASS = 'results-list'
ADDRESS_CLASS = 'property-card__address'
AREA_CLASS = 'property-card__detail-area'
ROOM_CLASS = 'property-card__detail-room'
BATHROOM_CLASS = 'property-card__detail-bathroom'
GARAGE_CLASS = 'property-card__detail-garage'
AMENITIES_CLASS = 'property-card__amenities'
PRICE_CLASS = 'property-card__price'
CONDO_CLASS = 'js-condo-price'
VALUE_CLASS = 'property-card__detail-value'
CHANGE_PAGE_XPATH = "//a[@title='Próxima página']"

In [ ]:
driver.get(VIVA_REAL_LINK)
full_results = []

num_pages = 200

for page in tqdm(range(num_pages)):
  #sleep necessario para garantir o carregamento da pagina
  sleep(6)

  #verificar se nao tem pop-up de cookie para aceitar
  try:
    driver.find_element_by_class_name("cookie-notifier__cta").click()
  except:
    print("No cookies!")

  #busca a lista de todos os resultados da pagina
  results_page = driver.find_element_by_class_name(RESULT_LIST_CLASS)
  #cria uma lista com todos os elementos que estao dentro de uma tag div e possuem o atributo data-type
  ids_list = results_page.find_elements_by_xpath('//div[@data-type]')
  for j in range(len(ids_list)):
    
    #coleta o id do ap
    url = ids_list[j].find_element_by_class_name('js-card-title').get_attribute('href')
    id = url.split("id-")[-1][:-1]

    #essa validacao existe para pular o card de anuncio que existe
    #caso seja ap, captura o endereco
    try:
      address = ids_list[j].find_element_by_class_name(ADDRESS_CLASS).text
    except:
      continue

    #recebe o elemento onde tem a informacao da area
    area_class = ids_list[j].find_element_by_class_name(AREA_CLASS)
    #captura apenas o numero da area
    area = area_class.find_element_by_class_name(VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de quartos
    room_class = ids_list[j].find_element_by_class_name(ROOM_CLASS)
    #captura apenas o numero de quartos
    room = room_class.find_element_by_class_name(VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de banheiros
    bathroom_class = ids_list[j].find_element_by_class_name(BATHROOM_CLASS)
    #captura apenas o numero de banheiros
    bathroom = bathroom_class.find_element_by_class_name(VALUE_CLASS).text

    #recebe o elemento onde tem a informacao de vagas
    garage_class = ids_list[j].find_element_by_class_name(GARAGE_CLASS)
    #captura apenas o numero de vagas
    garage = garage_class.find_element_by_class_name(VALUE_CLASS).text

    #verifica se existem extras e captura
    try:
      amenities = ids_list[j].find_element_by_class_name(AMENITIES_CLASS).text
    except:
      amenities = None
    
    #recebe o elemento onde tem a informacao do preco do aluguel
    price_class = ids_list[j].find_element_by_class_name(PRICE_CLASS)
    #captura o valor do aluguel
    price = price_class.find_element_by_tag_name('p').text

    #verifica se existe valor de condominio e captura
    try:
      condo = ids_list[j].find_element_by_class_name(CONDO_CLASS).text
    except:
      condo = None

    #adiciona um dicionario com todas as informacoes do ap na lista final
    full_results.append({'id': id,
                         'address': address, 
                         'area': area, 
                         'room': room, 
                         'bathroom': bathroom, 
                         'garage': garage, 
                         'amenities': amenities, 
                         'price': price, 
                         'condo': condo})
  
  #recebe o elemento onde tem a troca de página e clica
  next_page = driver.find_element_by_xpath(CHANGE_PAGE_XPATH)
  #print(next_page)
  next_page.click()

df = pd.DataFrame(full_results)
df.to_csv('full_results.csv', index=False)
driver.close()